In [30]:
# Dependencies
# import numpy as np
import pandas as pd
import datetime as dt
import requests
import json

# Extract: Data Pull

## Extract Data & Clean

### API Tracking data from CDC

#### Max temp days

In [31]:
# URL for GET requests to retrieve data
version = "v1"
measureId = 423 # Requires a valid measureID
stratificationLevelId = 82 # Requires a valid stratificationLevelId.
geographicTypeIdFilter = 1 # Requires a valid geographicTypeID or ALL.
geographicItemsFilter = 4 # Filter to retrieve only certain geographicItems.
temporalTypeIdFilter = 1 # Requires a valid temporal type id matching temporal items (e.g., 1)
temporalItemsFilter = "2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011" # Requires valid temporal items entries (e.g., 2020) separated by comma.
isSmoothed = 0 # 0 not smoothed- most data isn't
getFullCoreHolder = 0 # Requires either a one or zero (true or false) for fetching the full core holder. 


# Build the endpoint URL
MaxTempDays_url = f"https://ephtracking.cdc.gov/apigateway/api/{version}/getCoreHolder/{measureId}/{stratificationLevelId}/{geographicTypeIdFilter}/{geographicItemsFilter}/{temporalTypeIdFilter}/{temporalItemsFilter}/{isSmoothed}/{getFullCoreHolder}?TemperatureHeatIndexId=1&RelativeThresholdId=1" #{getFullCoreHolder}

response = requests.get(MaxTempDays_url).json()

# Isolate tableResult for easy reading
data = response["tableResult"]

#for index, element in enumerate(data):
#  print(index, ":", element)

In [32]:
# Create an empty list to hold summaries
summary_list = []

# loop through the data
for bob in data:
    county = bob["geo"]
    year = bob["temporal"]
    days = bob["dataValue"]
    
    # create summary dictionary
    summary_dict = {
        "county": county,
        "year": year,
        "ext_heat_days" : days
    }
    
    # Append the sumary dict to the list
    summary_list.append(summary_dict)

EHD = pd.DataFrame(summary_list) 
EHD

,county,year,ext_heat_days
0,Apache,2011,19
1,Apache,2012,19
2,Apache,2013,18
3,Apache,2014,5
4,Apache,2015,18
...,...,...,...
160,Yuma,2017,21
161,Yuma,2018,19
162,Yuma,2019,22
163,Yuma,2020,36


In [33]:
# Export the cleaned DataFrame as a CSV file. 
EHD.to_csv("data/ExtremeHeatDays.csv", encoding='utf8', index=False)

OSError: Cannot save file into a non-existent directory: 'data'

#### ED visits for over 35

In [ ]:
# URL for GET requests to retrieve data
version = "v1"
measureId = 108 # Requires a valid measureID
stratificationLevelId = 2 # Requires a valid stratificationLevelId.
geographicTypeIdFilter = "all" # Requires a valid geographicTypeID or ALL.
geographicItemsFilter = "all" # Filter to retrieve only certain geographicItems.
temporalTypeIdFilter = 1 # Requires a valid temporal type id matching temporal items (e.g., 1)
temporalItemsFilter = "2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011" # Requires valid temporal items entries (e.g., 2020) separated by comma.
isSmoothed = 0 # 0 not smoothed- most data isn't
getFullCoreHolder = 0 # Requires either a one or zero (true or false) for fetching the full core holder. 


# Build the endpoint URL
Hosp35_url = f"https://ephtracking.cdc.gov/apigateway/api/{version}/getCoreHolder/{measureId}/{stratificationLevelId}/{geographicTypeIdFilter}/{geographicItemsFilter}/{temporalTypeIdFilter}/{temporalItemsFilter}/{isSmoothed}/{getFullCoreHolder}"

#https://ephtracking.cdc.gov/apigateway/api/v1/getCoreHolder/108/2/all/all/1/2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011/0/0

response = requests.get(Hosp35_url).json()

# Isolate tableResult for easy reading
data = response["tableResult"]

In [ ]:
# Create an empty list to hold summaries
summary_list = []

# loop through the data
for bob in data:
    state = bob["parentGeoId"]
    county = bob["geo"]
    year = bob["temporal"]
    rate = bob["dataValue"]
    
    # create summary dictionary
    summary_dict = {
        "State" : state,
        "County": county,
        "Year": year,
        "Age Adjusted Rate ED" : rate
    }
    
    # Append the sumary dict to the list
    summary_list.append(summary_dict)

ED35 = pd.DataFrame(summary_list)  
ED35 #results all states

,State,County,Year,Age Adjusted Rate ED
0,04,Apache,2011,22.2
1,04,Apache,2012,25.9
2,04,Apache,2013,21.7
3,04,Apache,2014,17.5
4,04,Apache,2015,23.1
...,...,...,...,...
16308,55,Wood,2017,42.3
16309,55,Wood,2018,41.0
16310,55,Wood,2019,41.0
16311,55,Wood,2020,33.0


In [ ]:
#select rows where 'state' is AZ, 04
ED35 = ED35.loc[ED35['State'] == "04"]
ED35 = ED35.drop(columns=["State"])

In [ ]:
ED35.rename(columns={'County': 'county', 'Year': 'year', 'Age Adjusted Rate ED': 'ed_rate_35'}, inplace=True)
ED35.head()

,county,year,ed_rate_35
0,Apache,2011,22.2
1,Apache,2012,25.9
2,Apache,2013,21.7
3,Apache,2014,17.5
4,Apache,2015,23.1


In [ ]:
ED35.to_csv("data/EDRates35.csv", encoding='utf8', index=False)

### HRI Data from Arizona Department of Health Services (AZDHS)

#### ED visits

In [44]:
# File to Load
# Read the ED Visits data into a Pandas DataFrame
ed_visits_df = pd.read_csv('../data/AZ-EDVisits.csv')

# Get a brief summary of the ED visits DataFrame.
# ed_visits_df.info()

In [45]:
#drop duplicates & rows with GeogID 'ALL'
ed_visits_df = ed_visits_df.drop_duplicates(keep='first')
ed_visits_df = ed_visits_df[ed_visits_df['GeogID']!= 'ALL']
ed_visits_df.head()

,Name,Domain,Indicator,Year,GeogID,AgeGroup,Gender,Month,Measure,Value,ts,measureName,indicatorName,contentAreaName
0,NAVAJO,HSI,89,2011,4017,20,ALL,ALL,440,17.72,2023-04-18T10:21:46.240,Age-adjusted rate of emergency department visi...,Heat stress emergency department visits,Heat Stress Illness
2,PIMA,HSI,89,2011,4019,20,ALL,ALL,440,20.12,2023-04-18T10:21:46.240,Age-adjusted rate of emergency department visi...,Heat stress emergency department visits,Heat Stress Illness
4,PINAL,HSI,89,2011,4021,20,ALL,ALL,440,37.73,2023-04-18T10:21:46.240,Age-adjusted rate of emergency department visi...,Heat stress emergency department visits,Heat Stress Illness
6,SANTA CRUZ,HSI,89,2011,4023,20,ALL,ALL,440,17.72,2023-04-18T10:21:46.240,Age-adjusted rate of emergency department visi...,Heat stress emergency department visits,Heat Stress Illness
8,YAVAPAI,HSI,89,2011,4025,20,ALL,ALL,440,12.76,2023-04-18T10:21:46.240,Age-adjusted rate of emergency department visi...,Heat stress emergency department visits,Heat Stress Illness


In [46]:
#Pull Necessary columns and rename
ed_visits_df = ed_visits_df[['Name', 'Year', 'Value']]
ed_visits_df.rename(columns={'Name': 'county', 'Year': 'year', 'Value': 'ed_rate'}, inplace=True)
# ed_visits_df.head()

In [48]:
#sort and reset index
ed_visits_df = ed_visits_df.sort_values(by=['year', 'county'])
ed_visits_df = ed_visits_df.reset_index(drop=True)
ed_visits_df

,county,year,ed_rate
0,COCHISE,2011,23.72
1,COCONINO,2011,11.04
2,GILA,2011,29.87
3,GRAHAM,2011,38.41
4,LA PAZ,2011,144.96
...,...,...,...
144,PIMA,2021,22.65
145,PINAL,2021,42.39
146,SANTA CRUZ,2021,40.66
147,YAVAPAI,2021,28.35


In [ ]:
# Export the cleaned DataFrame as a CSV file. 
ed_visits_df.to_csv("data/cleaned-AZ-EDVisits.csv", encoding='utf8', index=False)

#### Hospitalization Data

In [50]:
# Read the Hospitalization data into a Pandas DataFrame
hosp_df = pd.read_csv('../data/AZ_Hospitalizations.csv')

# drop duplicates & rows with GeogID 'ALL'
hosp_df = hosp_df.drop_duplicates(keep='first')
hosp_df = hosp_df[hosp_df['GeogID']!= 'ALL']

# Pull Necessary columns and rename
hosp_df = hosp_df[['Name', 'Year', 'Value']]
hosp_df.rename(columns={'Name': 'county', 'Year': 'year', 'Value': 'hosp_rate'}, inplace=True)

# sort and reset index
hosp_df = hosp_df.sort_values(by=['year', 'county'])
hosp_df = hosp_df.reset_index(drop=True)

hosp_df

,county,year,hosp_rate
0,GILA,2011,0.00
1,GREENLEE,2011,0.00
2,MARICOPA,2011,8.73
3,MOHAVE,2011,11.43
4,PIMA,2011,4.79
...,...,...,...
82,PIMA,2021,8.65
83,PINAL,2021,9.93
84,SANTA CRUZ,2021,0.00
85,YAVAPAI,2021,2.81


In [ ]:
# Export the cleaned DataFrame as a CSV file. 
hosp_df.to_csv("data/cleaned_AZ_Hospitalizations.csv", encoding='utf8', index=False)

#### Vulnerability Data

In [52]:
# Read the Vulnerability data into a Pandas DataFrame
vulnerability_df = pd.read_csv('../data/AZ_Vulnerability.csv')

# drop duplicates & rows with GeogID 'ALL'
vulnerability_df = vulnerability_df.drop_duplicates(keep='first')
vulnerability_df = vulnerability_df[vulnerability_df['GeogID']!= 'ALL']

# Pull Necessary columns and rename
vulnerability_df = vulnerability_df[['Name', 'Year', 'Value']]
vulnerability_df.rename(columns={'Name': 'county', 'Year': 'year', 'Value': 'vul_rate'}, inplace=True)

# sort and reset index
vulnerability_df = vulnerability_df.sort_values(by=['year', 'county'])
vulnerability_df = vulnerability_df.reset_index(drop=True)
vulnerability_df

,county,year,vul_rate
0,APACHE,2010,0.993
1,COCHISE,2010,0.864
2,COCONINO,2010,0.819
3,GILA,2010,0.801
4,GRAHAM,2010,0.984
5,GREENLEE,2010,0.736
6,LA PAZ,2010,0.871
7,MARICOPA,2010,0.707
8,MOHAVE,2010,0.771
9,NAVAJO,2010,0.990


In [ ]:
# Export the cleaned DataFrame as a CSV file. 
vulnerability_df.to_csv("data/cleaned_AZ_Vulnerability.csv", encoding='utf8', index=False)

## Merge Data into CSV

In [57]:
# Combine the data into a single dataset.  
rates_df = pd.merge(ed_visits_df, hosp_df,how= "left",  on=['county','year'])
#rates_df

# Combine the data into a single dataset.  
df2 = pd.merge(rates_df, vulnerability_df, how= "left",  on=['county','year'])
df2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   county     149 non-null    object 
 1   year       149 non-null    int64  
 2   ed_rate    149 non-null    float64
 3   hosp_rate  76 non-null     float64
 4   vul_rate   25 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 7.0+ KB


In [59]:
# Export the cleaned DataFrame as a CSV file. 
df2.to_csv("../data/HRI.csv", encoding='utf8', index=False)

### Merge CSVs

In [60]:
# Read the HRI data into a Pandas DataFrame
HRI_df = pd.read_csv('../data/HRI.csv')
ExtHeat_df = pd.read_csv('../data/ExtremeHeatDays.csv')
ED35_df = pd.read_csv('../data/EDRates35.csv')

In [61]:
HRI_df

,county,year,ed_rate,hosp_rate,vul_rate
0,COCHISE,2011,23.72,NaN,NaN
1,COCONINO,2011,11.04,NaN,NaN
2,GILA,2011,29.87,0.00,NaN
3,GRAHAM,2011,38.41,NaN,NaN
4,LA PAZ,2011,144.96,NaN,NaN
...,...,...,...,...,...
144,PIMA,2021,22.65,8.65,NaN
145,PINAL,2021,42.39,9.93,NaN
146,SANTA CRUZ,2021,40.66,0.00,NaN
147,YAVAPAI,2021,28.35,2.81,NaN


In [62]:
# Merge all into one combined file
df3= pd.merge(HRI_df, ExtHeat_df, how= "left",  on=['county','year'])
df3

,county,year,ed_rate,hosp_rate,vul_rate,ext_heat_days
0,COCHISE,2011,23.72,NaN,NaN,NaN
1,COCONINO,2011,11.04,NaN,NaN,NaN
2,GILA,2011,29.87,0.00,NaN,NaN
3,GRAHAM,2011,38.41,NaN,NaN,NaN
4,LA PAZ,2011,144.96,NaN,NaN,NaN
...,...,...,...,...,...,...
144,PIMA,2021,22.65,8.65,NaN,NaN
145,PINAL,2021,42.39,9.93,NaN,NaN
146,SANTA CRUZ,2021,40.66,0.00,NaN,NaN
147,YAVAPAI,2021,28.35,2.81,NaN,NaN


In [63]:
# Merge all into one combined file
df4= pd.merge(df3, ED35_df, how= "left",  on=['county','year'])
df4

,county,year,ed_rate,hosp_rate,vul_rate,ext_heat_days,ed_rate_35
0,COCHISE,2011,23.72,NaN,NaN,NaN,NaN
1,COCONINO,2011,11.04,NaN,NaN,NaN,NaN
2,GILA,2011,29.87,0.00,NaN,NaN,NaN
3,GRAHAM,2011,38.41,NaN,NaN,NaN,NaN
4,LA PAZ,2011,144.96,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
144,PIMA,2021,22.65,8.65,NaN,NaN,NaN
145,PINAL,2021,42.39,9.93,NaN,NaN,NaN
146,SANTA CRUZ,2021,40.66,0.00,NaN,NaN,NaN
147,YAVAPAI,2021,28.35,2.81,NaN,NaN,NaN


In [67]:
df4['index'] = df4.index
print(df4)


         county  year  ed_rate  hosp_rate  vul_rate  ext_heat_days  \
0       COCHISE  2011    23.72        NaN       NaN            NaN   
1      COCONINO  2011    11.04        NaN       NaN            NaN   
2          GILA  2011    29.87       0.00       NaN            NaN   
3        GRAHAM  2011    38.41        NaN       NaN            NaN   
4        LA PAZ  2011   144.96        NaN       NaN            NaN   
..          ...   ...      ...        ...       ...            ...   
144        PIMA  2021    22.65       8.65       NaN            NaN   
145       PINAL  2021    42.39       9.93       NaN            NaN   
146  SANTA CRUZ  2021    40.66       0.00       NaN            NaN   
147     YAVAPAI  2021    28.35       2.81       NaN            NaN   
148        YUMA  2021    82.70      21.39       NaN            NaN   

     ed_rate_35  index  
0           NaN      0  
1           NaN      1  
2           NaN      2  
3           NaN      3  
4           NaN      4  
..       

In [69]:
df4.fillna(method='ffill', inplace=True)

In [70]:
print(df4)

         county  year  ed_rate  hosp_rate  vul_rate  ext_heat_days  \
0       COCHISE  2011    23.72        NaN       NaN            NaN   
1      COCONINO  2011    11.04        NaN       NaN            NaN   
2          GILA  2011    29.87       0.00       NaN            NaN   
3        GRAHAM  2011    38.41       0.00       NaN            NaN   
4        LA PAZ  2011   144.96       0.00       NaN            NaN   
..          ...   ...      ...        ...       ...            ...   
144        PIMA  2021    22.65       8.65      0.99            NaN   
145       PINAL  2021    42.39       9.93      0.99            NaN   
146  SANTA CRUZ  2021    40.66       0.00      0.99            NaN   
147     YAVAPAI  2021    28.35       2.81      0.99            NaN   
148        YUMA  2021    82.70      21.39      0.99            NaN   

     ed_rate_35  index  
0           NaN      0  
1           NaN      1  
2           NaN      2  
3           NaN      3  
4           NaN      4  
..       

In [73]:
# Export the cleaned DataFrame as a CSV file. 
#df4.to_csv("../data/AllHeat.csv", na_rep='NA', encoding='utf8', index=False)

# Load: SQLite database

In [65]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, distinct

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean

from pathlib import Path
import pandas as pd

In [8]:
Path('hri.sqlite').touch()

In [9]:
# create engine to HRI.sqlite
engine = create_engine("sqlite:///data/HRI.sqlite")

In [12]:
engine.execute('''CREATE TABLE vulnerability (county text, year int, vul_rate float)''')
engine.execute('''CREATE TABLE ed_visit_35 (county text, year int, ed_rate_35 float)''')

engine.execute('''CREATE TABLE hri (county text, year int, ED_rates float, Hosp_rates float, SVI float, HeatDays float)''')


OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [13]:
# load the data into a Pandas DataFrame and write to sqlite table
hri = pd.read_csv('../data/HRI.csv')
hri.to_sql('hri', engine, if_exists='append', index = False)

vulnerability = pd.read_csv('../data/cleaned_AZ_Vulnerability.csv')
vulnerability.to_sql('vulnerability', engine, if_exists='append', index = False)

ed_visit_35 = pd.read_csv('../data/EDRates35.csv')
ed_visit_35.to_sql('ed_visit_35', engine, if_exists='append', index = False)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)